In [1]:
import json
import codecs
import math
import re
import emoji

In [5]:
file_path = "H:\\Вова\\first-debate.json"
test_partition_path = "C:/Users/mrvol/Desktop/data_partition_0.json"

In [8]:
def get_tweet_count(file, partition_size=50000):
    #partition_size = 50000 # Будем партицировать по 50000 записей
    byte_position = 0
    with open(file, "r+", encoding='utf-8') as reader:
        string_count = 0
        while True:
            line  = reader.readline()
            if line != "":
                string_count += 1 
            else:
                break
        partitions_count = math.ceil(string_count/partition_size) #2027492
        for partition in range(partitions_count):
            with open(f"H:\\Вова\\Partitions\\first-debate_partition_{partition}.json", "w+", encoding='utf-8') as partition_writer:
                reader.seek(byte_position)
                for line in range(partition_size):
                    partition_writer.write(reader.readline())
                byte_position = reader.tell()

In [9]:
get_tweet_count(file_path)

In [6]:
def is_english(tweet_text):
    try:
        tweet_text.encode(encoding="utf-8").decode('ascii')
    except:
        return False
    else:
        return True

In [27]:
partition_size = 50000
for partition in range(41):#41
    with open("H/Вова/Partitions/data_partition_{}.json".format(partition), "r+", encoding="utf-8") as partition_reader:
        with open ("H/Вова/Text_partitions/text_partition_{}.txt".format(partition), "w+") as text_writer:
            #message_json = json.loads(partition_reader.readline())
            for line in range(partition_size):
                #text_writer.write( + "\n")
                tweet_text = json.loads(partition_reader.readline())["full_text"].replace("\n", ' ').replace("\r", ' ')
                if is_english(tweet_text):
                    tweet_text = re.sub(r'https://t.co/[A-Za-z0-9]+|[\.\?\\,!\(\);\-]', " ", tweet_text)
                    if tweet_text.startswith("RT"):
                        tweet_words = tweet_text.split(" ")[2:]
                    else:
                        tweet_words = tweet_text.split(" ")   
                    for word in tweet_words:
                        if word.endswith('…') or word == '' or word.endswith(":"):#:
                            tweet_words.remove(word)
                        for letter in list(word):
                            if letter in emoji.UNICODE_EMOJI:
                                word = ''.join(list(word).remove(letter))
                    text_writer.write(' '.join(tweet_words) + "\n")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Приступим к классификации
1 - Trump
2 - Hillary
3 - Поддерживает и следит за происходящим
4 - Против Trump
5 - Против Hillary
6 - Не относится к теме

In [33]:
word_vocabulary = []

group_array = [[], [], [], [], [], []]

test_data_path = "C:/Users/mrvol/Desktop/test_data.txt"

with open(test_data_path, "r+", encoding="utf-8") as test_data_reader:
    test_data = test_data_reader.readlines()

#Заполнили группы тестовыми данными
for line in test_data:
    line = line.split(",")
    print(line)
    group_array[int(line[1]) - 1].append(line[0])


["Any #debate that doesn't include @GovGaryJohnson is a sham #debatenight #Debates2016 #DebateDay #fb", ' 3\n']
['Ineed stark differences "We the People" need to hold all partisan elected officials from Prez down accountable for &amp to these platforms', ' 3\n']
['Watch for the BEST tonight Follow these guys #Debates #VoteTrump #TrumpTrain #MAGA', ' 1\n']
["Everything you need to know before watching tonight's", ' 3\n']
["A preview of tonight's debate #Debates2016 #Elections2016", ' 3 \n']
['Sigue nuestra cobertura #EnVivo del primer #debatenight en EU #Debates2016', ' 4\n']
['Everything you need to know about the first debate  #debates #debatenight', ' 3\n']
['The FBI\'s own stats So why do libs want open borders When will democrat voters &amp GOP cartel "lib enablers" wake up thx @CounterJihadUS', ' 3\n']
['#debatenight The whole debate time would be taken up if Hillary had to defend herself against all the scandals she has been involved in', ' 1\n']
["RT this if you've got Hillary's

In [34]:
text_vocabulary = []
# Заполняем массивы слов
for message_group in group_array:
    for text in message_group:
        for word in text.split(" "):
            if word.lower() not in text_vocabulary:
                text_vocabulary.append(word.lower())

In [48]:
group_vectors = [[], [], [], [], [], []]
# Создаем вектора из тестовых данных
for message_group in group_array:
    for text in message_group:
        text_vector = []
        for word in text_vocabulary:
            text_vector.append(1) if word in text else text_vector.append(0)
        group_vectors[group_array.index(message_group)].append(text_vector)

In [77]:
combined_vectors = []

import numpy
from numpy import linalg as la

for vector_group in group_vectors:
    group_vector = numpy.array(vector_group[0]) 
    for vector in vector_group[1:]:
        group_vector += numpy.array(vector)
    group_vector = vector_normalize(group_vector) 
    combined_vectors.append(group_vector)

combined_vectors

[[0.0,
  0.04824620135012933,
  0.0542769765188955,
  0.0,
  0.0542769765188955,
  0.0,
  0.012061550337532333,
  0.012061550337532333,
  0.036184651012597,
  0.0,
  0.0,
  0.0,
  0.036184651012597,
  0.006030775168766166,
  0.072369302025194,
  0.006030775168766166,
  0.006030775168766166,
  0.006030775168766166,
  0.006030775168766166,
  0.0180923255062985,
  0.012061550337532333,
  0.006030775168766166,
  0.006030775168766166,
  0.09046162753149249,
  0.006030775168766166,
  0.012061550337532333,
  0.006030775168766166,
  0.024123100675064665,
  0.006030775168766166,
  0.006030775168766166,
  0.0180923255062985,
  0.006030775168766166,
  0.006030775168766166,
  0.13870782888162184,
  0.006030775168766166,
  0.012061550337532333,
  0.0,
  0.012061550337532333,
  0.006030775168766166,
  0.006030775168766166,
  0.006030775168766166,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0542769765188955,
  0.012061550337532333,
  0.006030775168766166,
  0.0180923255062985,
  0.006030775168766166,
  0.0060307

In [70]:
def vector_normalize(vector):
    vector_length = 0
    for value in vector:
        vector_length += value**2        
    inversed_vector_length = 1/math.sqrt(vector_length)
    normalized_vector = []
    for value in vector:
        normalized_vector.append(value*inversed_vector_length)
    return normalized_vector

In [78]:
for group in group_array:
    print(f"Group - {group_array.index(group) + 1} - {len(group)/sum([len(element) for element in group_array])} - {len(group)}")

Group - 1 - 0.06431852986217458 - 42
Group - 2 - 0.13935681470137826 - 91
Group - 3 - 0.3675344563552833 - 240
Group - 4 - 0.10260336906584992 - 67
Group - 5 - 0.11179173047473201 - 73
Group - 6 - 0.21439509954058192 - 140


In [80]:
# import math

# vector_length = 0
# for num in group_vectors[0][0]:
#     vector_length += num
# print(math.sqrt(vector_length))
for partition in range(41):
    with open("C:/Users/mrvol/Desktop/text_partitions/text_partition_{}.txt".format(partition), "r+", encoding="utf-8") as partition_reader:
        vector_data = partition_reader.readlines()
        partition_vectors = []
        for line in vector_data:
            line_vector = []
            for word in text_vocabulary:
                line_vector.append(1) if word in line else line_vector.append(0)
            partition_vectors.append(line_vector)
            
        sorted_groups = [[], [], [], [], [], []]

        for vector in partition_vectors:
            vector = numpy.array(vector)
            vector_list = []
            for combined_vector in combined_vectors:
                vector_list.append(la.norm(vector - combined_vector))
            sorted_groups[vector_list.index(min(vector_list))].append(vector)
            
        print(f"Partition {partition}\n")
        for vector in sorted_groups:
            print(f"Group - {sorted_groups.index(vector) + 1} - {len(vector)/sum([len(vector) for vector in sorted_groups])}")


#         for sorted_group in sorted_groups:
#             print(len(sorted_group))
#             sorted_vector = numpy.array(sorted_group[0])
#             for vector in sorted_group[1:]:
#                 sorted_vector += numpy.array(vector)
#             combined_vectors[sorted_groups.index(sorted_group)] += sorted_vector
            

Partition 0

Group - 1 - 0.09781583909490886
Group - 2 - 0.09514456316781898
Group - 3 - 0.232950974230044
Group - 4 - 0.14919861722187303
Group - 5 - 0.19453174104336896
Group - 6 - 0.23035826524198616
Partition 1

Group - 1 - 0.0918785890073831
Group - 2 - 0.10023829055822493
Group - 3 - 0.2398531192624712
Group - 4 - 0.1338333528653463
Group - 5 - 0.17848353451306692
Group - 6 - 0.2557131137935076
Partition 2

Group - 1 - 0.09735389301351609
Group - 2 - 0.0924804873405673
Group - 3 - 0.2569960022844089
Group - 4 - 0.12960213211498192
Group - 5 - 0.16192651818008758
Group - 6 - 0.26164096706643825
Partition 3

Group - 1 - 0.10094441864065645
Group - 2 - 0.07698560148629818
Group - 3 - 0.25657996593899984
Group - 4 - 0.11975538008979718
Group - 5 - 0.16035764050162563
Group - 6 - 0.2853769933426227
Partition 4

Group - 1 - 0.1167825596692351
Group - 2 - 0.058823529411764705
Group - 3 - 0.24461567374553655
Group - 4 - 0.13320804360082691
Group - 5 - 0.16211238489005825
Group - 6 - 0.28

In [81]:
for vector in sorted_groups:
    print(f"Group - {sorted_groups.index(vector) + 1} - {len(vector)/sum([len(vector) for vector in sorted_groups])}")

Group - 1 - 0.12498680831603758
Group - 2 - 0.03538889084321244
Group - 3 - 0.15794842931016287
Group - 4 - 0.2034685334365216
Group - 5 - 0.1779997889330566
Group - 6 - 0.3002075491610089


In [157]:
with open(test_partition_path, "r+") as part_reader:
    json_value = json.loads(part_reader.readline())
    for key, value in json_value.items():
        print(str(key) + " - " + str(value) + "\r")
    # full_text
    # entities - hashtags - text
    # retweet_count
    

contributors - None
coordinates - None
created_at - Mon Sep 26 17:45:55 +0000 2016
display_text_range - [0, 139]
entities - {'hashtags': [{'indices': [27, 35], 'text': 'debates'}], 'symbols': [], 'urls': [], 'user_mentions': [{'id': 68707440, 'id_str': '68707440', 'indices': [3, 12], 'name': 'Cody Permenter', 'screen_name': 'somecody'}]}
favorite_count - 0
favorited - False
full_text - RT @somecody: Ahead of the #debates, can't even open up the Facebook Trending science and tech tab because there's nothing there. 

That's…
geo - None
id - 780463426501873700
id_str - 780463426501873664
in_reply_to_screen_name - None
in_reply_to_status_id - None
in_reply_to_status_id_str - None
in_reply_to_user_id - None
in_reply_to_user_id_str - None
is_quote_status - False
lang - en
place - None
retweet_count - 9
retweeted - False
retweeted_status - {'contributors': None, 'coordinates': None, 'created_at': 'Mon Sep 26 16:30:08 +0000 2016', 'display_text_range': [0, 133], 'entities': {'hashtags': [{'ind